In [1]:
import os
# demonstration of calculating metrics for a neural network model using sklearn
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

# example of random oversampling to balance the class distribution
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
from matplotlib import pyplot
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint

import tensorflow as tf

from scipy import stats

import datetime;
import warnings
warnings.filterwarnings("ignore")

2024-03-05 12:05:36.866421: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-05 12:05:37.526100: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-05 12:05:38.473376: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-03-05 12:05:38.473592: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [42]:
# configs
#EPOCHS = 5
BATCH_SIZE = 32

VERBOSE = 0

WEEKS_TEST= [3]
WEEKS_TEST= [2]
WEEKS_TRAIN = [0,1,2]
WEEKS_TRAIN = [0,1]


baseFolder = "../data_2019_processed/"
baseFolderWeek = baseFolder+"w"

fileSufixTrain = "_transformed" # unbalanced file sufix is empty -- _transformed
outputFileSufixTrain = "unb" # unbalanced file sufix is unb

# selected features
inputFeatures = ["activity","location","day_of_week",
                 "light","phone_lock","proximity",
                 "sound","time_to_next_alarm", "minutes_day"]
outputClasses = ["awake","asleep"]
#outputClasses = ["class"]

TIME_SERIES_SIZE = -1
TIME_STEP_SHIFT =  -1

NN_type = 'MLP'
UNITS_NUMBER = "16-8"

EPOCHS_ARRAY_TEST = [5,15,30,50,80,100,120,150,200]
EPOCHS_ARRAY_TEST = [1,5]
EPOCHS_ARRAY_TEST = [5,15,30,50,80,100]

generalName = "result_week_tested_"+str(NN_type)+"_"+outputFileSufixTrain+"_batch_size_"+str(BATCH_SIZE)+"_"+UNITS_NUMBER


outputMetricFile = generalName+".csv"
outputMetricFilePartials = generalName+"_partial.csv"
outputCheckpointFolder = generalName+"_checkpoints"
checkpointName_prefix = "checkpoint_epoch_{epoch}.hdf5"

In [43]:
print("Checking whether the checkpoint folder exists or not")
isExist = os.path.exists(outputCheckpointFolder)
if not isExist:
    # Create a new directory because it does not exist
    os.makedirs(outputCheckpointFolder)
    print("The new checkpoint directory is created!")
else:
    print("The checkpoint directory exists!")

Checking whether the checkpoint folder exists or not
The checkpoint directory exists!


In [44]:
outputMetricFile

'result_week_tested_MLP_unb_batch_size_32_16-8.csv'

In [45]:
# y_test     = Array with real values
# yhat_probs = Array with predicted values
def printMetrics(y_test,yhat_probs):
    # predict crisp classes for test set deprecated
    #yhat_classes = model.predict_classes(X_test, verbose=0)
    #yhat_classes = np.argmax(yhat_probs,axis=1)
    yhat_classes = yhat_probs.round()
    # accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(y_test, yhat_classes)
    print('Accuracy: %f' % accuracy)
    # precision tp / (tp + fp)
    precision = precision_score(y_test, yhat_classes)
    print('Precision: %f' % precision)
    # recall: tp / (tp + fn)
    recall = recall_score(y_test, yhat_classes)
    print('Recall: %f' % recall)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(y_test, yhat_classes)
    print('F1 score: %f' % f1)
    # kappa
    kappa = cohen_kappa_score(y_test, yhat_classes)
    print('Cohens kappa: %f' % kappa)
    # ROC AUC
    auc = roc_auc_score(y_test, yhat_probs)
    print('ROC AUC: %f' % auc)
    # confusion matrix
    print("\Confusion Matrix")
    matrix = confusion_matrix(y_test, yhat_classes)
    print(matrix)
    
    array = []
    results = dict()
    results['accuracy'] = accuracy
    results['precision'] = precision
    results['recall'] = recall
    results['f1_score'] = f1
    results['cohen_kappa_score'] = kappa
    results['roc_auc_score'] = auc
    #results['matrix'] = np.array(matrix,dtype=object)
    results['matrix'] = 0
    results['TP'] = matrix[0][0]
    results['FP'] = matrix[0][1]
    results['FN'] = matrix[1][0]
    results['TN'] = matrix[1][1]
    
    array.append(accuracy)
    array.append(precision)
    array.append(recall)
    array.append(f1)
    array.append(kappa)
    array.append(auc)
    #array.append(np.array(matrix,dtype=object)))
    array.append(0)
    array.append(matrix[0][0]) # TP
    array.append(matrix[0][1]) # FP
    array.append(matrix[1][0]) # FN
    array.append(matrix[1][1]) # TN
    
    return results, array

def showGlobalMetrics(metrics):
    accuracy,precision,recall,f1_score,cohen_kappa_score,roc_auc_score = 0,0,0,0,0,0
    for metric in metrics:
        accuracy = accuracy + metric['accuracy']
        precision = precision + metric['precision']
        recall = recall + metric['recall']
        f1_score = f1_score + metric['f1_score']
        cohen_kappa_score = cohen_kappa_score + metric['cohen_kappa_score']
        roc_auc_score = roc_auc_score + metric['roc_auc_score']
        
    # mean
    size = len(metrics)
    print(size)
    accuracy = accuracy / size
    precision = precision / size
    recall = recall / size
    f1_score = f1_score / size
    cohen_kappa_score = cohen_kappa_score / size
    roc_auc_score = roc_auc_score / size
    
    #show:\
    print("accuracy: ",accuracy)
    print("precision: ",precision)
    print("recall: ",recall)
    print("f1_score: ",f1_score)
    print("cohen_kappa_score: ",cohen_kappa_score)
    print("roc_auc_score: ",roc_auc_score)
    
    return [accuracy,precision,recall,f1_score,cohen_kappa_score,roc_auc_score]
    
def transform_data_type(dataframe):
    
    # transform inputs
    for column in inputFeatures:
        dataframe[column] = dataframe[column].astype('float32')
    
    # transform outputs
    for column in outputClasses:
        dataframe[column] = dataframe[column].astype('float32')
    
    return dataframe

# one-hot encoding function
def transform_output_nominal_class_into_one_hot_encoding(dataset):
    # create two classes based on the single class
    one_hot_encoded_data = pd.get_dummies(dataset['class'])
    #print(one_hot_encoded_data)
    dataset['awake'] = one_hot_encoded_data['awake']
    dataset['asleep'] = one_hot_encoded_data['asleep']
    
    return dataset

# one-hot encoding function
def transform_output_numerical_class_into_one_hot_encoding(dataset):
    # create two classes based on the single class
    one_hot_encoded_data = pd.get_dummies(dataset['class'])
    #print(one_hot_encoded_data)
    dataset['awake'] = one_hot_encoded_data[0]
    dataset['asleep'] = one_hot_encoded_data[1]
    
    return dataset


def create_dataset_time_series_with_one_output(X, y, window_time_steps=1, shift_step=1):
    Xs, ys = [], []
    for i in range(0, len(X) - window_time_steps, shift_step):
        v = X.iloc[i:(i + window_time_steps)].values
        labels = y.iloc[i: i + window_time_steps]
        Xs.append(v)        
        ys.append(stats.mode(labels)[0][0])
        
    if len(y.columns) == 1:
        return np.array(Xs), np.array(ys).reshape(-1, 1)
    else:
        return np.array(Xs), np.array(ys).reshape(-1, len(y.columns))
    
def create_dataset_time_series_with_one_output_foward(X, y, window_time_steps=1, shift_step=1):
    Xs, ys = [], []
    for i in range(0, len(X) - window_time_steps, shift_step):
        valuesX = X.iloc[i:(i + window_time_steps)].values # values
        valuesY = y.iloc[i: i + window_time_steps].values # labels
        
        Xs.append(valuesX)
        ys.append(valuesY[window_time_steps-1]) # append only the last value
        
    if len(y.columns) == 1:
        return np.array(Xs), np.array(ys).reshape(-1, 1)
    else:
        return np.array(Xs), np.array(ys).reshape(-1, len(y.columns))

In [46]:
# client datasets used on the training process (75% of data)
trainFolders =  ['0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs',
                '0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA', 
                '2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0', 
                '2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys', 
                #['5FLZBTVAPwdq9QezHE2sVCJIs7p+r6mCemA2gp9jATk'], #does not have the file
                '7EYF5I04EVqisUJCVNHlqn77UAuOmwL2Dahxd3cA', 
                'a9Qgj8ENWrHvl9QqlXcIPKmyGMKgbfHk9Dbqon1HQP4', 
                'ae4JJBZDycEcY8McJF+3BxyvZ1619y03BNdCxzpZTc', 
                'Ch3u5Oaz96VSrQbf0z31X6jEIbeIekkC0mwPzCdeJ1U', 
                'CH8f0yZkZL13zWuE9ks1CkVJRVrr+jsGdUXHrZ6YeA', 
                'DHO1K4jgiwZJOfQTrxvKE2vn7hkjamigroGD5IaeRc', 
                #'DHPqzSqSttiba1L3BD1cptNJPjSxZ8rXxF9mY3za6WA', # does not have asleep data
                'dQEFscjqnIlug8Tgq97JohhSQPG2DEOWJqS86wCrcY', 
                'HFvs2CohmhHte+AaCzFasjzegGzxZKPhkrX23iI6Xo', 
                'jgB9E8v3Z6PKdTRTCMAijBllA9YEMtrmHbe4qsbmJWw', 
                'JkY++R7E8myldLN3on6iQ78Ee78zCbrLuggfwGju3I', 
                'K4SLohf+TN1Ak8Dn8iE3Lme7rEMPISfppB2sXfHX8', 
                'oGaWetJJJEWHuvYdWYo826SQxfhCExVVQ2da8LE1Y7Q', 
                'pyt24oiDAHsmgWMvkFKz2fn2pwcHiXchd6KchLM', 
                #'PZCf1nfvhR+6fk+7+sPNMYOgb8BAMmtQtfoRS83Suc', # does not have asleep data
                'QUNCATForxzK0HHw46LrGOMWh0eVA8Y5XWEiUXX+cQ', 
                #'rIl2UK9+bQ+tzpFdbJAdbBxEa5GbgrgC030yEaENLw', 
                #'RoBW3cDOO9wWRMPO2twQff83MPc+OXn6gJ+a1DafreI', 
                'SH3kQeyd5volraxw8vOyhlowNqWBPr1IJ9URNXUL4']
                #'VVpwFNMrEglveh6MDN8lrRzTy5OwzglD4FURfM4A2is', 
                #'Wa1mcNmbh66S7VS6GIzyfCFMD3SGhbtDQyFP1ywJEsw', 
                #'XCKRE0BWRHxfP1kZIihgtT+jUjSp2GE8v5ZlhcIhVmA', 
                #'YI5Y79K6GXqAUoGP6PNyII8WKlAoel4urDxWSVVOvBw', 
                #'ypklj+8GJ15rOIH1lpKQtFJOuK+VdvyCuBPqhY3aoM', 
                #'ZSsAZ0Pq+MCqFrnjsRFn5Ua09pMCVaOV9c8ZuYb7XQY']
                
# client datasets used on the training process (75% of data)
trainFolders =  ['0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs',
                '0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA', 
                '2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0', 
                '2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys', 
                #['5FLZBTVAPwdq9QezHE2sVCJIs7p+r6mCemA2gp9jATk'], #does not have the file
                '7EYF5I04EVqisUJCVNHlqn77UAuOmwL2Dahxd3cA', 
                'a9Qgj8ENWrHvl9QqlXcIPKmyGMKgbfHk9Dbqon1HQP4', 
                'ae4JJBZDycEcY8McJF+3BxyvZ1619y03BNdCxzpZTc', 
                'Ch3u5Oaz96VSrQbf0z31X6jEIbeIekkC0mwPzCdeJ1U', 
                'CH8f0yZkZL13zWuE9ks1CkVJRVrr+jsGdUXHrZ6YeA', 
                'DHO1K4jgiwZJOfQTrxvKE2vn7hkjamigroGD5IaeRc', 
                #'DHPqzSqSttiba1L3BD1cptNJPjSxZ8rXxF9mY3za6WA', # does not have asleep data
                'dQEFscjqnIlug8Tgq97JohhSQPG2DEOWJqS86wCrcY', 
                'HFvs2CohmhHte+AaCzFasjzegGzxZKPhkrX23iI6Xo', 
                'jgB9E8v3Z6PKdTRTCMAijBllA9YEMtrmHbe4qsbmJWw', 
                'JkY++R7E8myldLN3on6iQ78Ee78zCbrLuggfwGju3I', 
                'K4SLohf+TN1Ak8Dn8iE3Lme7rEMPISfppB2sXfHX8', 
                'oGaWetJJJEWHuvYdWYo826SQxfhCExVVQ2da8LE1Y7Q', 
                'pyt24oiDAHsmgWMvkFKz2fn2pwcHiXchd6KchLM', 
                #'PZCf1nfvhR+6fk+7+sPNMYOgb8BAMmtQtfoRS83Suc', # does not have asleep data
                'QUNCATForxzK0HHw46LrGOMWh0eVA8Y5XWEiUXX+cQ', 
                'rIl2UK9+bQ+tzpFdbJAdbBxEa5GbgrgC030yEaENLw', 
                'RoBW3cDOO9wWRMPO2twQff83MPc+OXn6gJ+a1DafreI', 
                'SH3kQeyd5volraxw8vOyhlowNqWBPr1IJ9URNXUL4',
                'VVpwFNMrEglveh6MDN8lrRzTy5OwzglD4FURfM4A2is', 
                'Wa1mcNmbh66S7VS6GIzyfCFMD3SGhbtDQyFP1ywJEsw', 
                'XCKRE0BWRHxfP1kZIihgtT+jUjSp2GE8v5ZlhcIhVmA', 
                'YI5Y79K6GXqAUoGP6PNyII8WKlAoel4urDxWSVVOvBw', 
                'ypklj+8GJ15rOIH1lpKQtFJOuK+VdvyCuBPqhY3aoM', 
                'ZSsAZ0Pq+MCqFrnjsRFn5Ua09pMCVaOV9c8ZuYb7XQY']
            
# client datasets used on the training process (25% of data)
testFolders =  [#'0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs',
                #'0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA', 
                #'2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0', 
                #'2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys', 
                #['5FLZBTVAPwdq9QezHE2sVCJIs7p+r6mCemA2gp9jATk'], #does not have the file
                #'7EYF5I04EVqisUJCVNHlqn77UAuOmwL2Dahxd3cA', 
                #'a9Qgj8ENWrHvl9QqlXcIPKmyGMKgbfHk9Dbqon1HQP4', 
                #'ae4JJBZDycEcY8McJF+3BxyvZ1619y03BNdCxzpZTc', 
                #'Ch3u5Oaz96VSrQbf0z31X6jEIbeIekkC0mwPzCdeJ1U', 
                #'CH8f0yZkZL13zWuE9ks1CkVJRVrr+jsGdUXHrZ6YeA', 
                #'DHO1K4jgiwZJOfQTrxvKE2vn7hkjamigroGD5IaeRc', 
                #'DHPqzSqSttiba1L3BD1cptNJPjSxZ8rXxF9mY3za6WA', # does not have asleep data
                #'dQEFscjqnIlug8Tgq97JohhSQPG2DEOWJqS86wCrcY', 
                #'HFvs2CohmhHte+AaCzFasjzegGzxZKPhkrX23iI6Xo', 
                #'jgB9E8v3Z6PKdTRTCMAijBllA9YEMtrmHbe4qsbmJWw', 
                #'JkY++R7E8myldLN3on6iQ78Ee78zCbrLuggfwGju3I', 
                #'K4SLohf+TN1Ak8Dn8iE3Lme7rEMPISfppB2sXfHX8', 
                #'oGaWetJJJEWHuvYdWYo826SQxfhCExVVQ2da8LE1Y7Q', 
                #'pyt24oiDAHsmgWMvkFKz2fn2pwcHiXchd6KchLM', 
                #'PZCf1nfvhR+6fk+7+sPNMYOgb8BAMmtQtfoRS83Suc', # does not have asleep data
                #'QUNCATForxzK0HHw46LrGOMWh0eVA8Y5XWEiUXX+cQ', 
                'rIl2UK9+bQ+tzpFdbJAdbBxEa5GbgrgC030yEaENLw', 
                'RoBW3cDOO9wWRMPO2twQff83MPc+OXn6gJ+a1DafreI', 
                #'SH3kQeyd5volraxw8vOyhlowNqWBPr1IJ9URNXUL4'] 
                'VVpwFNMrEglveh6MDN8lrRzTy5OwzglD4FURfM4A2is', 
                'Wa1mcNmbh66S7VS6GIzyfCFMD3SGhbtDQyFP1ywJEsw', 
                'XCKRE0BWRHxfP1kZIihgtT+jUjSp2GE8v5ZlhcIhVmA', 
                'YI5Y79K6GXqAUoGP6PNyII8WKlAoel4urDxWSVVOvBw', 
                'ypklj+8GJ15rOIH1lpKQtFJOuK+VdvyCuBPqhY3aoM', 
                'ZSsAZ0Pq+MCqFrnjsRFn5Ua09pMCVaOV9c8ZuYb7XQY']

# take the list of directories and concat them
def loadDataFromFolders(foldersToLoad,inputFolders,fileType = "_transformed"):
    print(len(foldersToLoad), "datasets")
    for i in range(0,len(foldersToLoad)):
        currentFolder = foldersToLoad[i]
        print(i , "-", currentFolder,inputFolders+"/student_"+currentFolder+""+fileType+".csv")
        #print(trainingDataSet[i])
        if(i == 0):
            temp_data = pd.read_csv(inputFolders+"/student_"+currentFolder+""+fileType+".csv")
        else:
            dataset = pd.read_csv(inputFolders+"/student_"+currentFolder+""+fileType+".csv")
            temp_data = pd.concat([temp_data, dataset])
    # return the dataset        
    return temp_data

In [47]:
print("Preparing test data")

# WEEKS_TEST= [3]
# WEEKS_TRAIN = [0,1,2]
# initialize
print("Initializing with first week data")

folderToLoad = baseFolderWeek + str(WEEKS_TEST[0]) + "/"
X_test = loadDataFromFolders(trainFolders,folderToLoad,"_transformed")

X_test2 = loadDataFromFolders(testFolders,baseFolder,"_transformed")

for week_value in WEEKS_TEST:

    if week_value != WEEKS_TEST[0]:
        
        
        print("Loading week:",week_value)
        
        folderToLoad = baseFolderWeek + str(week_value) + "/"

        # test data comprising 25% of the data. It must be fixed to all models being evaluated
        #X_test  = pd.read_csv(inputFolders+"test/allData-classification-numeric-normalized.csv")
        temp_data = loadDataFromFolders(trainFolders,folderToLoad,"_transformed")

        X_test = pd.concat([temp_data, X_test])
        print()
        # undestand the dataset by looking on their infos
        print(X_test.info())

X_test

Preparing test data
Initializing with first week data
27 datasets
0 - 0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs ../data_2019_processed/w2//student_0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs_transformed.csv
1 - 0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA ../data_2019_processed/w2//student_0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA_transformed.csv
2 - 2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0 ../data_2019_processed/w2//student_2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0_transformed.csv
3 - 2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys ../data_2019_processed/w2//student_2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys_transformed.csv
4 - 7EYF5I04EVqisUJCVNHlqn77UAuOmwL2Dahxd3cA ../data_2019_processed/w2//student_7EYF5I04EVqisUJCVNHlqn77UAuOmwL2Dahxd3cA_transformed.csv
5 - a9Qgj8ENWrHvl9QqlXcIPKmyGMKgbfHk9Dbqon1HQP4 ../data_2019_processed/w2//student_a9Qgj8ENWrHvl9QqlXcIPKmyGMKgbfHk9Dbqon1HQP4_transformed.csv
6 - ae4JJBZDycEcY8McJF+3BxyvZ1619y03BNdCxzpZTc ../data_2019_processed/w2//student_ae4JJBZD

,activity,location,timestamp,time_to_next_alarm,sound,proximity,phone_lock,light,day_of_week,minutes_day,timestamp_text,class
0,0.00,0.0,0.000000e+00,0.969841,0.000000,1.0,0.0,0.000207,0.666667,0.714385,2018-05-26 17:08:23+00:00,awake
1,0.25,1.0,0.000000e+00,0.969841,0.616077,1.0,1.0,0.000384,0.666667,0.714385,2018-05-26 17:08:53+00:00,awake
2,0.25,1.0,0.000000e+00,0.969742,0.616077,1.0,1.0,0.000384,0.666667,0.715080,2018-05-26 17:09:23+00:00,awake
3,0.25,1.0,3.219683e-07,0.969643,0.723862,1.0,1.0,0.000384,0.666667,0.715775,2018-05-26 17:10:54+00:00,awake
4,0.25,1.0,3.219683e-07,0.969544,0.723862,1.0,1.0,0.000384,0.666667,0.716470,2018-05-26 17:11:24+00:00,awake
...,...,...,...,...,...,...,...,...,...,...,...,...
5219,0.75,0.5,9.156778e-04,0.929068,0.539918,0.0,1.0,0.000030,0.166667,0.927728,2018-05-30 22:15:43+00:00,awake
5220,0.00,0.5,9.163217e-04,0.928571,0.537753,0.0,1.0,0.000000,0.166667,0.931202,2018-05-30 22:20:44+00:00,awake
5221,0.00,0.5,9.166437e-04,0.928472,0.555459,0.0,1.0,0.000000,0.166667,0.931897,2018-05-30 22:21:44+00:00,awake
5222,0.75,0.0,9.172877e-04,0.927976,0.571294,0.0,1.0,0.000000,0.166667,0.935372,2018-05-30 22:26:45+00:00,awake


In [48]:
X_test2

,activity,location,timestamp,time_to_next_alarm,sound,proximity,phone_lock,light,day_of_week,minutes_day,timestamp_text,class
0,0.75,1.0,0.000000e+00,0.000000,0.515992,1.0,0.0,0.000000,1.0,0.678249,2018-05-14 16:16:08+00:00,awake
1,0.25,1.0,3.211282e-07,0.000000,0.542171,0.0,1.0,0.000007,1.0,0.678944,2018-05-14 16:17:39+00:00,awake
2,0.25,1.0,6.422564e-07,0.000000,0.515992,0.0,1.0,0.000000,1.0,0.679639,2018-05-14 16:18:39+00:00,awake
3,0.00,1.0,6.422564e-07,0.000000,0.515992,0.0,1.0,0.000000,1.0,0.680334,2018-05-14 16:19:09+00:00,awake
4,0.25,1.0,6.422564e-07,0.000000,0.531341,0.0,1.0,0.000000,1.0,0.681028,2018-05-14 16:20:09+00:00,awake
...,...,...,...,...,...,...,...,...,...,...,...,...
13208,1.00,0.0,3.981050e-03,0.923909,0.577338,1.0,0.0,0.000266,1.0,0.035441,2018-06-05 00:51:21+00:00,awake
13209,1.00,0.0,3.981050e-03,0.923809,0.612109,1.0,0.0,0.000236,1.0,0.036136,2018-06-05 00:52:21+00:00,awake
13210,0.00,0.0,3.981371e-03,0.923611,0.573415,1.0,0.0,0.000118,1.0,0.037526,2018-06-05 00:54:11+00:00,awake
13211,0.75,0.0,3.981693e-03,0.923611,0.538685,1.0,0.0,0.000089,1.0,0.037526,2018-06-05 00:54:41+00:00,awake


In [49]:
print("Preparing X_train data")
# WEEKS_TEST= [3]
# WEEKS_TRAIN = [0,1,2]
# initialize
print("Initializing with first week data")

folderToLoad = baseFolderWeek + str(WEEKS_TRAIN[0]) + "/"
X_train = loadDataFromFolders(testFolders,folderToLoad,"_transformed")

for week_value in WEEKS_TRAIN:

    if week_value != WEEKS_TRAIN[0]:
        
        
        print("Loading week:",week_value)
        
        folderToLoad = baseFolderWeek + str(week_value) + "/"

        # test data comprising 25% of the data. It must be fixed to all models being evaluated
        #X_test  = pd.read_csv(inputFolders+"test/allData-classification-numeric-normalized.csv")
        temp_data = loadDataFromFolders(trainFolders,folderToLoad,"_transformed")

        X_train = pd.concat([temp_data, X_test])
        print()
        # undestand the dataset by looking on their infos
        print(X_train.info())

X_train

Preparing X_train data
Initializing with first week data
8 datasets
0 - rIl2UK9+bQ+tzpFdbJAdbBxEa5GbgrgC030yEaENLw ../data_2019_processed/w0//student_rIl2UK9+bQ+tzpFdbJAdbBxEa5GbgrgC030yEaENLw_transformed.csv
1 - RoBW3cDOO9wWRMPO2twQff83MPc+OXn6gJ+a1DafreI ../data_2019_processed/w0//student_RoBW3cDOO9wWRMPO2twQff83MPc+OXn6gJ+a1DafreI_transformed.csv
2 - VVpwFNMrEglveh6MDN8lrRzTy5OwzglD4FURfM4A2is ../data_2019_processed/w0//student_VVpwFNMrEglveh6MDN8lrRzTy5OwzglD4FURfM4A2is_transformed.csv
3 - Wa1mcNmbh66S7VS6GIzyfCFMD3SGhbtDQyFP1ywJEsw ../data_2019_processed/w0//student_Wa1mcNmbh66S7VS6GIzyfCFMD3SGhbtDQyFP1ywJEsw_transformed.csv
4 - XCKRE0BWRHxfP1kZIihgtT+jUjSp2GE8v5ZlhcIhVmA ../data_2019_processed/w0//student_XCKRE0BWRHxfP1kZIihgtT+jUjSp2GE8v5ZlhcIhVmA_transformed.csv
5 - YI5Y79K6GXqAUoGP6PNyII8WKlAoel4urDxWSVVOvBw ../data_2019_processed/w0//student_YI5Y79K6GXqAUoGP6PNyII8WKlAoel4urDxWSVVOvBw_transformed.csv
6 - ypklj+8GJ15rOIH1lpKQtFJOuK+VdvyCuBPqhY3aoM ../data_2019_processed/w0//st

,activity,location,timestamp,time_to_next_alarm,sound,proximity,phone_lock,light,day_of_week,minutes_day,timestamp_text,class
0,1.00,0.0,0.000000e+00,0.987996,0.569082,0.0,1.0,0.000000,0.500000,0.587213,2018-05-18 14:05:01+00:00,awake
1,1.00,0.0,6.428012e-07,0.987698,0.560956,0.0,1.0,0.000034,0.500000,0.589298,2018-05-18 14:08:32+00:00,awake
2,1.00,0.0,9.642017e-07,0.987500,0.580815,0.0,1.0,0.000067,0.500000,0.591383,2018-05-18 14:11:02+00:00,awake
3,1.00,0.0,9.642017e-07,0.987401,0.563893,0.0,1.0,0.000000,0.500000,0.591383,2018-05-18 14:11:32+00:00,awake
4,0.00,0.0,1.607003e-06,0.987004,0.565485,0.0,1.0,0.000000,0.500000,0.594163,2018-05-18 14:15:41+00:00,awake
...,...,...,...,...,...,...,...,...,...,...,...,...
5219,0.75,0.5,9.156778e-04,0.929068,0.539918,0.0,1.0,0.000030,0.166667,0.927728,2018-05-30 22:15:43+00:00,awake
5220,0.00,0.5,9.163217e-04,0.928571,0.537753,0.0,1.0,0.000000,0.166667,0.931202,2018-05-30 22:20:44+00:00,awake
5221,0.00,0.5,9.166437e-04,0.928472,0.555459,0.0,1.0,0.000000,0.166667,0.931897,2018-05-30 22:21:44+00:00,awake
5222,0.75,0.0,9.172877e-04,0.927976,0.571294,0.0,1.0,0.000000,0.166667,0.935372,2018-05-30 22:26:45+00:00,awake


In [50]:
#X_train = pd.read_csv(baseFolder+"train/allData-classification-numeric-normalized.csv")
#X_test  = pd.read_csv(baseFolder+"test/allData-classification-numeric-normalized.csv")
#X_train = pd.read_csv(baseFolder+"train/allData-classification-numeric-normalized_balanced_undersample.csv")
#X_test  = pd.read_csv(baseFolder+"test/allData-classification-numeric-normalized_balanced_oversample.csv")

#AA = pd.read_csv(baseFolder+"allData-classification-numeric-normalized.csv")
#X_train, X_test = train_test_split(AA,test_size=0.25)

In [51]:
print(X_train.info())
X_train

<class 'pandas.core.frame.DataFrame'>
Index: 308097 entries, 0 to 5223
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   activity            308097 non-null  float64
 1   location            308097 non-null  float64
 2   timestamp           308097 non-null  float64
 3   time_to_next_alarm  308097 non-null  float64
 4   sound               308097 non-null  float64
 5   proximity           308097 non-null  float64
 6   phone_lock          308097 non-null  float64
 7   light               308097 non-null  float64
 8   day_of_week         308097 non-null  float64
 9   minutes_day         308097 non-null  float64
 10  timestamp_text      308097 non-null  object 
 11  class               308097 non-null  object 
dtypes: float64(10), object(2)
memory usage: 30.6+ MB
None


,activity,location,timestamp,time_to_next_alarm,sound,proximity,phone_lock,light,day_of_week,minutes_day,timestamp_text,class
0,1.00,0.0,0.000000e+00,0.987996,0.569082,0.0,1.0,0.000000,0.500000,0.587213,2018-05-18 14:05:01+00:00,awake
1,1.00,0.0,6.428012e-07,0.987698,0.560956,0.0,1.0,0.000034,0.500000,0.589298,2018-05-18 14:08:32+00:00,awake
2,1.00,0.0,9.642017e-07,0.987500,0.580815,0.0,1.0,0.000067,0.500000,0.591383,2018-05-18 14:11:02+00:00,awake
3,1.00,0.0,9.642017e-07,0.987401,0.563893,0.0,1.0,0.000000,0.500000,0.591383,2018-05-18 14:11:32+00:00,awake
4,0.00,0.0,1.607003e-06,0.987004,0.565485,0.0,1.0,0.000000,0.500000,0.594163,2018-05-18 14:15:41+00:00,awake
...,...,...,...,...,...,...,...,...,...,...,...,...
5219,0.75,0.5,9.156778e-04,0.929068,0.539918,0.0,1.0,0.000030,0.166667,0.927728,2018-05-30 22:15:43+00:00,awake
5220,0.00,0.5,9.163217e-04,0.928571,0.537753,0.0,1.0,0.000000,0.166667,0.931202,2018-05-30 22:20:44+00:00,awake
5221,0.00,0.5,9.166437e-04,0.928472,0.555459,0.0,1.0,0.000000,0.166667,0.931897,2018-05-30 22:21:44+00:00,awake
5222,0.75,0.0,9.172877e-04,0.927976,0.571294,0.0,1.0,0.000000,0.166667,0.935372,2018-05-30 22:26:45+00:00,awake


In [52]:
# transform output to one_hot_encoding for the testing dataset
X_test = transform_output_nominal_class_into_one_hot_encoding(X_test)

# transform output to one_hot_encoding for the testing dataset
X_test2 = transform_output_nominal_class_into_one_hot_encoding(X_test2)

# transform output to one_hot_encoding for the testing dataset
X_train = transform_output_nominal_class_into_one_hot_encoding(X_train)


# transforms the input data to float32
X_test = transform_data_type(X_test)

# transforms the input data to float32
X_test2 = transform_data_type(X_test2)

# transforms the input data to float32
X_train = transform_data_type(X_train)

In [53]:
X_train

,activity,location,timestamp,time_to_next_alarm,sound,proximity,phone_lock,light,day_of_week,minutes_day,timestamp_text,class,awake,asleep
0,1.00,0.0,0.000000e+00,0.987996,0.569082,0.0,1.0,0.000000,0.500000,0.587213,2018-05-18 14:05:01+00:00,awake,1.0,0.0
1,1.00,0.0,6.428012e-07,0.987698,0.560956,0.0,1.0,0.000034,0.500000,0.589298,2018-05-18 14:08:32+00:00,awake,1.0,0.0
2,1.00,0.0,9.642017e-07,0.987500,0.580815,0.0,1.0,0.000067,0.500000,0.591383,2018-05-18 14:11:02+00:00,awake,1.0,0.0
3,1.00,0.0,9.642017e-07,0.987401,0.563893,0.0,1.0,0.000000,0.500000,0.591383,2018-05-18 14:11:32+00:00,awake,1.0,0.0
4,0.00,0.0,1.607003e-06,0.987004,0.565485,0.0,1.0,0.000000,0.500000,0.594163,2018-05-18 14:15:41+00:00,awake,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5219,0.75,0.5,9.156778e-04,0.929067,0.539918,0.0,1.0,0.000030,0.166667,0.927728,2018-05-30 22:15:43+00:00,awake,1.0,0.0
5220,0.00,0.5,9.163217e-04,0.928571,0.537753,0.0,1.0,0.000000,0.166667,0.931202,2018-05-30 22:20:44+00:00,awake,1.0,0.0
5221,0.00,0.5,9.166437e-04,0.928472,0.555459,0.0,1.0,0.000000,0.166667,0.931897,2018-05-30 22:21:44+00:00,awake,1.0,0.0
5222,0.75,0.0,9.172877e-04,0.927976,0.571294,0.0,1.0,0.000000,0.166667,0.935372,2018-05-30 22:26:45+00:00,awake,1.0,0.0


In [54]:
# selects the data to train and test
X_train_data = pd.DataFrame(data=X_train,columns=inputFeatures)
y_train_data = pd.DataFrame(data=X_train,columns=outputClasses)
# selec test dataset (fixed to all)
X_test_data = pd.DataFrame(data=X_test,columns=inputFeatures)
y_test_data = pd.DataFrame(data=X_test,columns=outputClasses)

# selec test dataset (fixed to all)
X_test2_data = pd.DataFrame(data=X_test2,columns=inputFeatures)
y_test2_data = pd.DataFrame(data=X_test2,columns=outputClasses)

In [55]:
X_test_data

,activity,location,day_of_week,light,phone_lock,proximity,sound,time_to_next_alarm,minutes_day
0,0.00,0.0,0.666667,0.000207,0.0,1.0,0.000000,0.969841,0.714385
1,0.25,1.0,0.666667,0.000384,1.0,1.0,0.616077,0.969841,0.714385
2,0.25,1.0,0.666667,0.000384,1.0,1.0,0.616077,0.969742,0.715080
3,0.25,1.0,0.666667,0.000384,1.0,1.0,0.723862,0.969643,0.715775
4,0.25,1.0,0.666667,0.000384,1.0,1.0,0.723862,0.969544,0.716470
...,...,...,...,...,...,...,...,...,...
5219,0.75,0.5,0.166667,0.000030,1.0,0.0,0.539918,0.929067,0.927728
5220,0.00,0.5,0.166667,0.000000,1.0,0.0,0.537753,0.928571,0.931202
5221,0.00,0.5,0.166667,0.000000,1.0,0.0,0.555459,0.928472,0.931897
5222,0.75,0.0,0.166667,0.000000,1.0,0.0,0.571294,0.927976,0.935372


In [56]:
y_test2_data

,awake,asleep
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0
...,...,...
13208,1.0,0.0
13209,1.0,0.0
13210,1.0,0.0
13211,1.0,0.0


In [57]:
# transtorm data to tensor slices
test_dataset_slice = tf.data.Dataset.from_tensor_slices((X_test_data, y_test_data))
test2_dataset_slice = tf.data.Dataset.from_tensor_slices((X_test2_data, y_test2_data))
train_dataset_slice = tf.data.Dataset.from_tensor_slices((X_train_data, y_train_data))

In [58]:
# batch_size
train_dataset_batch = train_dataset_slice.batch(BATCH_SIZE)

train_dataset_batch

<BatchDataset element_spec=(TensorSpec(shape=(None, 9), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))>

In [59]:
# batch_size
validation_dataset_batch = test_dataset_slice.batch(BATCH_SIZE)

validation_dataset_batch

<BatchDataset element_spec=(TensorSpec(shape=(None, 9), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))>

In [60]:
MAX_EPOCH = max(EPOCHS_ARRAY_TEST)

MAX_EPOCH

100

In [61]:
print("configure checkpoint")
filepath = outputCheckpointFolder + "/" + checkpointName_prefix
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=VERBOSE, save_best_only=False, mode='auto')
#WARNING:tensorflow:`period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.


configure checkpoint


In [62]:
#generate model
model = keras.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=(len(inputFeatures),)))
model.add(tf.keras.layers.Dense(len(inputFeatures)))
model.add(tf.keras.layers.Dense(16, activation=tf.keras.activations.relu))
model.add(tf.keras.layers.Dense(8, activation=tf.keras.activations.relu),)
model.add(tf.keras.layers.Dense(len(outputClasses), activation='softmax'))

# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])
          #loss='binary_crossentropy',loss='categorical_crossentropy',
          #loss='binary_crossentropy',  sparse_categorical_crossentropy 
        
print("input_shape=[", X_train_data.shape[0], X_train_data.shape[1],"]")
print("output shape:",len(outputClasses))
print("Epochs:",MAX_EPOCH)

current_time = datetime.datetime.now()
time_stamp = current_time.timestamp()
print("Start timestamp:", time_stamp,current_time)
print()

verbose, epochs, batch_size = VERBOSE, MAX_EPOCH, BATCH_SIZE  
# fit network
history = model.fit(train_dataset_batch, 
                    epochs=epochs, 
                    verbose=verbose, 
                    batch_size=batch_size, 
                    callbacks=[checkpoint],
                    validation_data=validation_dataset_batch) #, batch_size=batch_size, validation_split=0.1

# generate time metrics
current_time2 = datetime.datetime.now()
time_stamp2 = current_time2.timestamp()
processing_time_s = (time_stamp2-time_stamp)
print("End timestamp:", time_stamp2,current_time2)
print("Processing time (s):", (processing_time_s))
print("Processing time (m):", (processing_time_s/60))
print("Processing time (h):", ((processing_time_s/60)/60))

input_shape=[ 308097 9 ]
output shape: 2
Epochs: 100
Start timestamp: 1709651834.458075 2024-03-05 12:17:14.458075

End timestamp: 1709653197.079314 2024-03-05 12:39:57.079314
Processing time (s): 1362.6212389469147
Processing time (m): 22.710353982448577
Processing time (h): 0.3785058997074763


In [63]:
dfhistory = pd.DataFrame(data=history.history) 

print("print all loss, ",len(dfhistory))
history

outputHistoryFilepath = outputCheckpointFolder+"/train_metrics_history.csv"
print("save all loss on ",outputHistoryFilepath)
dfhistory.to_csv(outputHistoryFilepath, sep=',', encoding='utf-8', index=False)

# outputCheckpointFolder
#history.history.to_csv(outputMetricFile, sep=',', encoding='utf-8', index=False)
# loss and val_loss
# acc and val_acc
history.history["loss"]
history.history["categorical_accuracy"]

dfhistory

print all loss,  100
save all loss on  result_week_tested_MLP_unb_batch_size_32_16-8_checkpoints/train_metrics_history.csv


,loss,categorical_accuracy,val_loss,val_categorical_accuracy
0,0.491299,0.758459,0.257244,0.871363
1,0.403723,0.799797,0.222638,0.899502
2,0.344536,0.860498,0.203967,0.919793
3,0.323997,0.878279,0.185954,0.950230
4,0.411333,0.802958,0.238158,0.904288
...,...,...,...,...
95,0.281354,0.854614,0.174614,0.911945
96,0.288255,0.849414,0.183097,0.907734
97,0.305983,0.841349,0.180788,0.907734
98,0.274937,0.861307,0.182725,0.905628


In [64]:
columnsOutputMetrics = ['NN_type','units','epochs','batch_size','window_size','time_step_shift',
           'start_time','end_time','time_s','time_m', "train_accuracy", "train_loss", "val_accuracy", "val_loss",
           'class','accuracy','precision','recall','f1_score','cohen_kappa_score','roc_auc_score','confusion_matrix',
           'TP','FP','FN','TN']

allMetrics = []

In [65]:
#generate model
model = keras.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=(len(inputFeatures),)))
model.add(tf.keras.layers.Dense(len(inputFeatures)))
model.add(tf.keras.layers.Dense(16, activation=tf.keras.activations.relu))
model.add(tf.keras.layers.Dense(8, activation=tf.keras.activations.relu),)
model.add(tf.keras.layers.Dense(len(outputClasses), activation='softmax'))

# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])
          #loss='binary_crossentropy',loss='categorical_crossentropy',
          #loss='binary_crossentropy',  sparse_categorical_crossentropy 

for TEST_EPOCHS in EPOCHS_ARRAY_TEST:
    # general data from the run
    generalData = [NN_type,UNITS_NUMBER,TEST_EPOCHS,BATCH_SIZE,TIME_SERIES_SIZE,TIME_STEP_SHIFT]
    
    print("input_shape=[", X_train_data.shape,"]")
    print("output shape:",len(outputClasses))
    print("Epochs:",TEST_EPOCHS)
    
    current_time = datetime.datetime.now()
    time_stamp = current_time.timestamp()
    print("Start timestamp:", time_stamp,current_time)
    print()
    
    checkPpath = filepath.replace("{epoch}",str(TEST_EPOCHS))
    print("Loading checkpoint: ", checkPpath)
    print('')
    
    model.load_weights(checkPpath)
    # evaluate model
    #accuracy = model.evaluate(test_dataset_series1) # , batch_size=batch_size, verbose=0
    # predict
    yhat_probs = model.predict(X_test_data,verbose=VERBOSE)
    # predict crisp classes for test set deprecated
    
    # generate time metrics
    current_time2 = datetime.datetime.now()
    time_stamp2 = current_time2.timestamp()
    processing_time_s = (time_stamp2-time_stamp)
    
    train_accuracy = history.history["loss"][TEST_EPOCHS-1]
    train_loss     = history.history["categorical_accuracy"][TEST_EPOCHS-1]
    val_acc = history.history["val_loss"][TEST_EPOCHS-1]
    val_loss     = history.history["val_categorical_accuracy"][TEST_EPOCHS-1]
    # generate general metrics
    rowData = [current_time,current_time2,processing_time_s,(processing_time_s)/60,train_accuracy,train_loss,val_acc,val_loss]

    y_pred_labels = pd.DataFrame(data=yhat_probs,columns=['awake','asleep'])
    y_test_labels = pd.DataFrame(data=y_test_data,columns=['awake','asleep'])

    feature_metrics_gathered = []
    # print('')
    print('awake')    
    res,resA = printMetrics(y_test_labels['awake'],y_pred_labels['awake'])
    feature_metrics_gathered.append(res)
   
    #columns = ['NN_type','units','epochs','batch_size','max_iterations',''Users',
    #            round_iteration','start_time','end_time','round_time_s','round_time_m',
    #           'class','accuracy','precision','recall','f1_score','cohen_kappa_score','roc_auc_score','confusion_matrix',
    #           'TP','FP','FN','TN']
    # new data
    classData = np.concatenate((['awake'], resA))
    classData = np.concatenate((rowData, classData))
    classData = np.concatenate((generalData, classData))
    allMetrics.append(classData)
    
    print('')
    print('asleep')
    res,resA = printMetrics(y_test_labels['asleep'],y_pred_labels['asleep'])
    feature_metrics_gathered.append(res)
    # new data
    classData = np.concatenate((['asleep'], resA))
    classData = np.concatenate((rowData, classData))
    classData = np.concatenate((generalData, classData))
    allMetrics.append(classData)
    print('')
    print('Global')
    resA = showGlobalMetrics(feature_metrics_gathered) #return [accuracy,precision,recall,f1_score,cohen_kappa_score,roc_auc_score
    # new data
    classData = np.concatenate((['avg'], resA))
    classData = np.concatenate((rowData, classData))
    classData = np.concatenate((generalData, classData))
    allMetrics.append(classData)
    print('')
    print("End timestamp:", time_stamp2,current_time2)
    print('')
    print('')
    print('-----------------------------------------------------------------------')
    print('')
    print('')

input_shape=[ (308097, 9) ]
output shape: 2
Epochs: 5
Start timestamp: 1709653197.575877 2024-03-05 12:39:57.575877

Loading checkpoint:  result_week_tested_MLP_unb_batch_size_32_16-8_checkpoints/checkpoint_epoch_5.hdf5

awake
Accuracy: 0.904288
Precision: 0.977505
Recall: 0.887769
F1 score: 0.930478
Cohens kappa: 0.778050
ROC AUC: 0.951694
\Confusion Matrix
[[37206  2079]
 [11421 90342]]

asleep
Accuracy: 0.904288
Precision: 0.765130
Recall: 0.947079
F1 score: 0.846437
Cohens kappa: 0.778050
ROC AUC: 0.951694
\Confusion Matrix
[[90342 11421]
 [ 2079 37206]]

Global
2
accuracy:  0.9042879019908117
precision:  0.8713177977697008
recall:  0.9174238383484731
f1_score:  0.8884578278349134
cohen_kappa_score:  0.778050333629776
roc_auc_score:  0.9516938470976612

End timestamp: 1709653201.449421 2024-03-05 12:40:01.449421


-----------------------------------------------------------------------


input_shape=[ (308097, 9) ]
output shape: 2
Epochs: 15
Start timestamp: 1709653201.918299 2024-0

In [66]:
dataMetrics = pd.DataFrame(data=allMetrics,columns=columnsOutputMetrics) 

dataMetrics

,NN_type,units,epochs,batch_size,window_size,time_step_shift,start_time,end_time,time_s,time_m,...,precision,recall,f1_score,cohen_kappa_score,roc_auc_score,confusion_matrix,TP,FP,FN,TN
0,MLP,16-8,5,32,-1,-1,2024-03-05 12:39:57.575877,2024-03-05 12:40:01.449421,3.873544,0.064559,...,0.9775051124744376,0.8877686388962589,0.9304783092324804,0.778050333629776,0.9516938470976614,0.0,37206.0,2079.0,11421.0,90342.0
1,MLP,16-8,5,32,-1,-1,2024-03-05 12:39:57.575877,2024-03-05 12:40:01.449421,3.873544,0.064559,...,0.7651304830649639,0.9470790378006873,0.8464373464373465,0.778050333629776,0.9516938470976612,0.0,90342.0,11421.0,2079.0,37206.0
2,MLP,16-8,5,32,-1,-1,2024-03-05 12:39:57.575877,2024-03-05 12:40:01.449421,3.873544,0.064559,...,0.8713177977697008,0.9174238383484731,0.8884578278349134,0.778050333629776,0.9516938470976612,None,None,None,None,None
3,MLP,16-8,15,32,-1,-1,2024-03-05 12:40:01.918299,2024-03-05 12:40:05.301735,3.383436,0.056391,...,0.8491506228765572,0.9946935526664897,0.916177908113392,0.6216948771797074,0.8833507935509342,0.0,21303.0,17982.0,540.0,101223.0
4,MLP,16-8,15,32,-1,-1,2024-03-05 12:40:01.918299,2024-03-05 12:40:05.301735,3.383436,0.056391,...,0.9752781211372065,0.5422680412371134,0.6969964664310954,0.6216948771797074,0.8833507935509343,0.0,101223.0,540.0,17982.0,21303.0
5,MLP,16-8,15,32,-1,-1,2024-03-05 12:40:01.918299,2024-03-05 12:40:05.301735,3.383436,0.056391,...,0.9122143720068818,0.7684807969518015,0.8065871872722437,0.6216948771797074,0.8833507935509342,None,None,None,None,None
6,MLP,16-8,30,32,-1,-1,2024-03-05 12:40:05.781956,2024-03-05 12:40:09.180644,3.398688,0.056645,...,0.9630044843049327,0.9116476518970549,0.936622597792013,0.7875290477380026,0.9687539840988257,0.0,35721.0,3564.0,8991.0,92772.0
7,MLP,16-8,30,32,-1,-1,2024-03-05 12:40:05.781956,2024-03-05 12:40:09.180644,3.398688,0.056645,...,0.7989130434782609,0.9092783505154639,0.8505303760848602,0.7875290477380026,0.9687543488031097,0.0,92772.0,8991.0,3564.0,35721.0
8,MLP,16-8,30,32,-1,-1,2024-03-05 12:40:05.781956,2024-03-05 12:40:09.180644,3.398688,0.056645,...,0.8809587638915968,0.9104630012062593,0.8935764869384366,0.7875290477380026,0.9687541664509678,None,None,None,None,None
9,MLP,16-8,50,32,-1,-1,2024-03-05 12:40:09.642987,2024-03-05 12:40:13.061648,3.418661,0.056978,...,0.9863801452784504,0.8646855929954895,0.9215325887176588,0.7592686693568476,0.9631183438724428,0.0,38070.0,1215.0,13770.0,87993.0


In [67]:
dataMetrics.to_csv(outputMetricFile, sep=',', encoding='utf-8', index=False)

In [68]:
#generate model
model = keras.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=(len(inputFeatures),)))
model.add(tf.keras.layers.Dense(len(inputFeatures)))
model.add(tf.keras.layers.Dense(16, activation=tf.keras.activations.relu))
model.add(tf.keras.layers.Dense(8, activation=tf.keras.activations.relu),)
model.add(tf.keras.layers.Dense(len(outputClasses), activation='softmax'))

# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])
          #loss='binary_crossentropy',loss='categorical_crossentropy',
          #loss='binary_crossentropy',  sparse_categorical_crossentropy 

for TEST_EPOCHS in EPOCHS_ARRAY_TEST:
    # general data from the run
    generalData = [NN_type,UNITS_NUMBER,TEST_EPOCHS,BATCH_SIZE,TIME_SERIES_SIZE,TIME_STEP_SHIFT]
    
    print("input_shape=[", X_train_data.shape,"]")
    print("output shape:",len(outputClasses))
    print("Epochs:",TEST_EPOCHS)
    
    current_time = datetime.datetime.now()
    time_stamp = current_time.timestamp()
    print("Start timestamp:", time_stamp,current_time)
    print()
    
    checkPpath = filepath.replace("{epoch}",str(TEST_EPOCHS))
    print("Loading checkpoint: ", checkPpath)
    print('')
    
    model.load_weights(checkPpath)
    # evaluate model
    #accuracy = model.evaluate(test_dataset_series1) # , batch_size=batch_size, verbose=0
    # predict
    yhat_probs = model.predict(X_test2_data,verbose=VERBOSE)
    # predict crisp classes for test set deprecated
    
    # generate time metrics
    current_time2 = datetime.datetime.now()
    time_stamp2 = current_time2.timestamp()
    processing_time_s = (time_stamp2-time_stamp)
    
    train_accuracy = history.history["loss"][TEST_EPOCHS-1]
    train_loss     = history.history["categorical_accuracy"][TEST_EPOCHS-1]
    val_acc = history.history["val_loss"][TEST_EPOCHS-1]
    val_loss     = history.history["val_categorical_accuracy"][TEST_EPOCHS-1]
    # generate general metrics
    rowData = [current_time,current_time2,processing_time_s,(processing_time_s)/60,train_accuracy,train_loss,val_acc,val_loss]

    y_pred_labels = pd.DataFrame(data=yhat_probs,columns=['awake','asleep'])
    y_test_labels = pd.DataFrame(data=y_test2_data,columns=['awake','asleep'])

    feature_metrics_gathered = []
    # print('')
    print('awake')    
    res,resA = printMetrics(y_test_labels['awake'],y_pred_labels['awake'])
    feature_metrics_gathered.append(res)
   
    #columns = ['NN_type','units','epochs','batch_size','max_iterations',''Users',
    #            round_iteration','start_time','end_time','round_time_s','round_time_m',
    #           'class','accuracy','precision','recall','f1_score','cohen_kappa_score','roc_auc_score','confusion_matrix',
    #           'TP','FP','FN','TN']
    # new data
    classData = np.concatenate((['awake'], resA))
    classData = np.concatenate((rowData, classData))
    classData = np.concatenate((generalData, classData))
    allMetrics.append(classData)
    
    print('')
    print('asleep')
    res,resA = printMetrics(y_test_labels['asleep'],y_pred_labels['asleep'])
    feature_metrics_gathered.append(res)
    # new data
    classData = np.concatenate((['asleep'], resA))
    classData = np.concatenate((rowData, classData))
    classData = np.concatenate((generalData, classData))
    allMetrics.append(classData)
    print('')
    print('Global')
    resA = showGlobalMetrics(feature_metrics_gathered) #return [accuracy,precision,recall,f1_score,cohen_kappa_score,roc_auc_score
    # new data
    classData = np.concatenate((['avg'], resA))
    classData = np.concatenate((rowData, classData))
    classData = np.concatenate((generalData, classData))
    allMetrics.append(classData)
    print('')
    print("End timestamp:", time_stamp2,current_time2)
    print('')
    print('')
    print('-----------------------------------------------------------------------')
    print('')
    print('')

input_shape=[ (308097, 9) ]
output shape: 2
Epochs: 5
Start timestamp: 1709653221.359499 2024-03-05 12:40:21.359499

Loading checkpoint:  result_week_tested_MLP_unb_batch_size_32_16-8_checkpoints/checkpoint_epoch_5.hdf5

awake
Accuracy: 0.662450
Precision: 0.872864
Recall: 0.659996
F1 score: 0.751650
Cohens kappa: 0.255370
ROC AUC: 0.709784
\Confusion Matrix
[[16772  8229]
 [29105 56497]]

asleep
Accuracy: 0.662450
Precision: 0.365586
Recall: 0.670853
F1 score: 0.473264
Cohens kappa: 0.255370
ROC AUC: 0.709784
\Confusion Matrix
[[56497 29105]
 [ 8229 16772]]

Global
2
accuracy:  0.6624503856134101
precision:  0.6192251573208698
recall:  0.665424713821475
f1_score:  0.612456822107373
cohen_kappa_score:  0.25537039309466825
roc_auc_score:  0.7097837735517472

End timestamp: 1709653224.044481 2024-03-05 12:40:24.044481


-----------------------------------------------------------------------


input_shape=[ (308097, 9) ]
output shape: 2
Epochs: 15
Start timestamp: 1709653224.414914 2024-0

In [69]:
dataMetrics = pd.DataFrame(data=allMetrics,columns=columnsOutputMetrics) 

dataMetrics

,NN_type,units,epochs,batch_size,window_size,time_step_shift,start_time,end_time,time_s,time_m,...,precision,recall,f1_score,cohen_kappa_score,roc_auc_score,confusion_matrix,TP,FP,FN,TN
0,MLP,16-8,5,32,-1,-1,2024-03-05 12:39:57.575877,2024-03-05 12:40:01.449421,3.873544,0.064559,...,0.9775051124744376,0.8877686388962589,0.9304783092324804,0.778050333629776,0.9516938470976614,0.0,37206.0,2079.0,11421.0,90342.0
1,MLP,16-8,5,32,-1,-1,2024-03-05 12:39:57.575877,2024-03-05 12:40:01.449421,3.873544,0.064559,...,0.7651304830649639,0.9470790378006873,0.8464373464373465,0.778050333629776,0.9516938470976612,0.0,90342.0,11421.0,2079.0,37206.0
2,MLP,16-8,5,32,-1,-1,2024-03-05 12:39:57.575877,2024-03-05 12:40:01.449421,3.873544,0.064559,...,0.8713177977697008,0.9174238383484731,0.8884578278349134,0.778050333629776,0.9516938470976612,None,None,None,None,None
3,MLP,16-8,15,32,-1,-1,2024-03-05 12:40:01.918299,2024-03-05 12:40:05.301735,3.383436,0.056391,...,0.8491506228765572,0.9946935526664897,0.916177908113392,0.6216948771797074,0.8833507935509342,0.0,21303.0,17982.0,540.0,101223.0
4,MLP,16-8,15,32,-1,-1,2024-03-05 12:40:01.918299,2024-03-05 12:40:05.301735,3.383436,0.056391,...,0.9752781211372065,0.5422680412371134,0.6969964664310954,0.6216948771797074,0.8833507935509343,0.0,101223.0,540.0,17982.0,21303.0
5,MLP,16-8,15,32,-1,-1,2024-03-05 12:40:01.918299,2024-03-05 12:40:05.301735,3.383436,0.056391,...,0.9122143720068818,0.7684807969518015,0.8065871872722437,0.6216948771797074,0.8833507935509342,None,None,None,None,None
6,MLP,16-8,30,32,-1,-1,2024-03-05 12:40:05.781956,2024-03-05 12:40:09.180644,3.398688,0.056645,...,0.9630044843049327,0.9116476518970549,0.936622597792013,0.7875290477380026,0.9687539840988257,0.0,35721.0,3564.0,8991.0,92772.0
7,MLP,16-8,30,32,-1,-1,2024-03-05 12:40:05.781956,2024-03-05 12:40:09.180644,3.398688,0.056645,...,0.7989130434782609,0.9092783505154639,0.8505303760848602,0.7875290477380026,0.9687543488031097,0.0,92772.0,8991.0,3564.0,35721.0
8,MLP,16-8,30,32,-1,-1,2024-03-05 12:40:05.781956,2024-03-05 12:40:09.180644,3.398688,0.056645,...,0.8809587638915968,0.9104630012062593,0.8935764869384366,0.7875290477380026,0.9687541664509678,None,None,None,None,None
9,MLP,16-8,50,32,-1,-1,2024-03-05 12:40:09.642987,2024-03-05 12:40:13.061648,3.418661,0.056978,...,0.9863801452784504,0.8646855929954895,0.9215325887176588,0.7592686693568476,0.9631183438724428,0.0,38070.0,1215.0,13770.0,87993.0


In [70]:
dataMetrics.to_csv(outputMetricFile+"2", sep=',', encoding='utf-8', index=False)

In [71]:
model.get_weights()

[array([[-0.20320967, -1.0102483 ,  0.05739519, -1.0757353 , -2.160995  ,
         -1.5086793 , -1.639359  ,  0.44533184, -0.05810731],
        [ 0.44744778,  1.1514125 ,  0.84481925, -0.9932074 ,  1.7540832 ,
         -0.02835491, -0.8140008 , -1.4742191 ,  1.4595127 ],
        [-6.8972783 , -0.22276601, -0.8275189 ,  2.242105  , -2.5895042 ,
         -0.95937115,  0.63894403, -1.6793087 ,  1.5716499 ],
        [-0.9836259 , -1.5310779 ,  0.70945835, -2.326464  , -1.4789162 ,
         -2.5729241 ,  3.7326808 ,  2.5890868 ,  4.410094  ],
        [-0.11101218, -0.19422413, -0.9084119 , -1.2712251 ,  0.27071527,
          2.024329  ,  0.43205333, -0.3411542 , -0.42030522],
        [-0.3817431 , -0.47676352, -1.5456724 , -1.2888615 ,  0.75843024,
          2.3745666 , -1.0572356 , -1.1174152 , -1.7456675 ],
        [ 0.54913193, -1.5280645 , -0.53917783,  0.5738059 , -1.7430472 ,
         -1.4084421 , -0.2524143 ,  1.4682962 ,  0.05843651],
        [-0.16838409, -4.8333244 , -0.15046252, 

In [72]:
model_json = model.to_json()
model_json

'{"class_name": "Sequential", "config": {"name": "sequential_6", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 9], "dtype": "float32", "sparse": false, "ragged": false, "name": "input_7"}}, {"class_name": "Dense", "config": {"name": "dense_24", "trainable": true, "dtype": "float32", "units": 9, "activation": "linear", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dense", "config": {"name": "dense_25", "trainable": true, "dtype": "float32", "units": 16, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_

In [73]:
def save_weights_only(weights, filename):
    np.savez(filename, *weights)

def load_weights_only(filename):
    return np.load(filename, allow_pickle=True)

# Save only the weights
weights = model.get_weights()
save_weights_only(weights, 'model_weights.npz')

In [74]:
# Load the weights
loaded_weights = load_weights_only('model_weights.npz')
loaded_weights


NpzFile 'model_weights.npz' with keys: arr_0, arr_1, arr_2, arr_3, arr_4...

In [75]:
weights

[array([[-0.20320967, -1.0102483 ,  0.05739519, -1.0757353 , -2.160995  ,
         -1.5086793 , -1.639359  ,  0.44533184, -0.05810731],
        [ 0.44744778,  1.1514125 ,  0.84481925, -0.9932074 ,  1.7540832 ,
         -0.02835491, -0.8140008 , -1.4742191 ,  1.4595127 ],
        [-6.8972783 , -0.22276601, -0.8275189 ,  2.242105  , -2.5895042 ,
         -0.95937115,  0.63894403, -1.6793087 ,  1.5716499 ],
        [-0.9836259 , -1.5310779 ,  0.70945835, -2.326464  , -1.4789162 ,
         -2.5729241 ,  3.7326808 ,  2.5890868 ,  4.410094  ],
        [-0.11101218, -0.19422413, -0.9084119 , -1.2712251 ,  0.27071527,
          2.024329  ,  0.43205333, -0.3411542 , -0.42030522],
        [-0.3817431 , -0.47676352, -1.5456724 , -1.2888615 ,  0.75843024,
          2.3745666 , -1.0572356 , -1.1174152 , -1.7456675 ],
        [ 0.54913193, -1.5280645 , -0.53917783,  0.5738059 , -1.7430472 ,
         -1.4084421 , -0.2524143 ,  1.4682962 ,  0.05843651],
        [-0.16838409, -4.8333244 , -0.15046252, 

In [76]:
loaded_weights

NpzFile 'model_weights.npz' with keys: arr_0, arr_1, arr_2, arr_3, arr_4...

In [77]:
len(loaded_weights)

8

In [78]:
keys = loaded_weights.keys()
keys
arrays = []
for x in keys:
    print(x)
    
    arrays.append(loaded_weights[x])

arrays

arr_0
arr_1
arr_2
arr_3
arr_4
arr_5
arr_6
arr_7


[array([[-0.20320967, -1.0102483 ,  0.05739519, -1.0757353 , -2.160995  ,
         -1.5086793 , -1.639359  ,  0.44533184, -0.05810731],
        [ 0.44744778,  1.1514125 ,  0.84481925, -0.9932074 ,  1.7540832 ,
         -0.02835491, -0.8140008 , -1.4742191 ,  1.4595127 ],
        [-6.8972783 , -0.22276601, -0.8275189 ,  2.242105  , -2.5895042 ,
         -0.95937115,  0.63894403, -1.6793087 ,  1.5716499 ],
        [-0.9836259 , -1.5310779 ,  0.70945835, -2.326464  , -1.4789162 ,
         -2.5729241 ,  3.7326808 ,  2.5890868 ,  4.410094  ],
        [-0.11101218, -0.19422413, -0.9084119 , -1.2712251 ,  0.27071527,
          2.024329  ,  0.43205333, -0.3411542 , -0.42030522],
        [-0.3817431 , -0.47676352, -1.5456724 , -1.2888615 ,  0.75843024,
          2.3745666 , -1.0572356 , -1.1174152 , -1.7456675 ],
        [ 0.54913193, -1.5280645 , -0.53917783,  0.5738059 , -1.7430472 ,
         -1.4084421 , -0.2524143 ,  1.4682962 ,  0.05843651],
        [-0.16838409, -4.8333244 , -0.15046252, 

In [79]:
model2 = keras.Sequential()
model2.add(tf.keras.layers.InputLayer(input_shape=(len(inputFeatures),)))
model2.add(tf.keras.layers.Dense(len(inputFeatures)))
model2.add(tf.keras.layers.Dense(16, activation=tf.keras.activations.relu))
model2.add(tf.keras.layers.Dense(8, activation=tf.keras.activations.relu),)
model2.add(tf.keras.layers.Dense(len(outputClasses), activation='softmax'))

# compile model
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])
          #loss='binary_crossentropy',loss='categorical_crossentropy',
          #loss='binary_crossentropy',  sparse_categorical_crossentropy 

In [80]:
model2.set_weights(arrays)

In [81]:
model2